In [1]:
import numpy as np 
import pandas as pd
import sys
from astropy.io import fits
import plotly.graph_objects as go
sys.path.append('../../dash_gaia_orbits/')

# Package imports
from dash_gaia_orbits.dash_test import create_dash_app
from dash_gaia_orbits import StarClusterData, StarClusterCollection, StarClusters3DPlotter

A new version of galpy (1.9.2) is available, please upgrade using pip/conda/... to get the latest features and bug fixes!


In [2]:
# basic custom layout properties
width = 800
height = 800
x_width = 700
y_width = x_width
z_width = 400
z_aspect = 1.*(z_width/x_width)

In [3]:
plotly_template = 'plotly_dark'
color = 'white' if plotly_template == 'plotly_dark' else 'black'

In [4]:
# define custom layout
        # plotly scene, layout, and camera
camera = dict(
    up=dict(x=0, y=1, z=0),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0., y=-.8, z=1.5),
    projection = dict(type = 'perspective')
) 

scene = dict(
    camera = camera,
    aspectmode = 'manual',
    aspectratio = dict(x=1, y=1, z=z_aspect),
    xaxis=dict(
        title='X\' (pc)', 
        range = [-x_width, x_width],
        showgrid=False,
        gridcolor = color,
        zeroline=True,
        zerolinecolor = color,
        showline = True,
        mirror = True,
        showspikes = False,
        nticks = 5,
        linecolor = color,
        linewidth = 2.,
    ), 
    yaxis=dict(
        title='Y\' (pc)', 
        range = [-y_width, y_width],
        showgrid=False,
        gridcolor = color,
        zeroline=True,
        zerolinecolor = color,
        showline = True,
        mirror = True,
        showspikes = False,
        nticks = 5,
        linecolor = color,
        linewidth = 2.,
    ), 
    zaxis=dict(
        title='Z\' (pc)', 
        range = [-z_width, z_width],
        showgrid=False,
        gridcolor = color,
        zeroline=True,
        zerolinecolor = color,
        showline = True,
        showspikes = False,
        mirror = True,
        nticks = 5,
        linecolor = color,
        linewidth = 2.,
    )
)

legend = dict(
    x = 0,
    y = 1,
    title = dict(
        text = 'Click to toggle traces on/off',
        font = dict(
            size = 18,  
            color = color
        ),  
        side = 'top'
    ),
    font = dict(
        size = 14,
        family = 'Helvetica',
        color = color
    ),
    itemsizing = 'constant',
    bgcolor = 'rgba(0,0,0,0)' # transparent legend
)

layout = dict(
    scene = scene,
    template = plotly_template, 
    dragmode = 'turntable',
    legend = legend,
    width = width,
    height = height
)

In [5]:
static_traces = []
static_traces_times = []


# read in Gordian scatter map
dfv = pd.read_csv('/Users/cam/Downloads/edenhofer_dust_scatter_new.csv')
#dfv = dfv.loc[dfv['extinction'].between(0.0003, 1)]
dfv = dfv.loc[dfv['extinction'].between(0.0005, 1)]
ds_index = 2
scatter_edenhofer = go.Scatter3d(
    x=dfv['X'].values[::ds_index],
    y=dfv['Y'].values[::ds_index],
    z=dfv['Z'].values[::ds_index],
    mode='markers',
    marker=dict(size=4.,
                #color='gray',
                color=dfv['extinction'].values[::ds_index],
                colorscale='Gray',
                cmin = 0.0005,
                cmax = 0.005,
                symbol='circle',
                opacity=.04),
    line = dict(color = 'gray', width = 0.),
    name='Edenhofer Dust',
    visible = True,
    hovertext='Edenhofer Dust',
    hoverinfo='skip'
    )
static_traces.append(scatter_edenhofer)
static_traces_times.append([0.]) # only show at t=0


hdu = fits.open('/Users/cam/Downloads/Local_Bubble_Pelgrims2020.fits')
x = hdu[1].data['x'][::4]
y = hdu[1].data['y'][::4]
z = hdu[1].data['z'][::4]

scatter_lb = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(size=1,
                color='#6495ED',
                symbol='circle',
                opacity=.05),
    name='Local Bubble',
    visible = 'legendonly',
    hovertext='Local Bubble')
static_traces.append(scatter_lb)
static_traces_times.append([0.]) # only show at t=0

In [6]:
df_hunt_gagne = pd.read_csv('/Users/cam/Downloads/cluster_sample_data.csv')
# limit to only necessary columns
df_hunt_gagne = df_hunt_gagne[['name', 'family', 'age_myr', 'U', 'V', 'W', 'x_helio', 'y_helio', 'z_helio', 'n_stars']]
# rename columns if necessary to be consistent with the package
df_hunt_gagne = df_hunt_gagne.rename(columns={'x_helio' : 'x', 'y_helio' : 'y', 'z_helio' : 'z'})

m6 = df_hunt_gagne[df_hunt_gagne['family'] == 'm6']
cr135 = df_hunt_gagne[df_hunt_gagne['family'] == 'cr135']
ap = df_hunt_gagne[df_hunt_gagne['family'] == 'alphaPer']

df_hunt_partIII = pd.read_csv('/Users/cam/Downloads/hunt_part3_clusters.csv')
df_hunt_partIII_to_merge = df_hunt_partIII[['name', 'kind']]
m6 = m6.merge(df_hunt_partIII_to_merge, on='name', how='left')

sun = pd.DataFrame({'name' : ['Sun'], 'family' : ['Sun'], 'age_myr' : [8000], 'U' : [0.], 'V' : [0.], 'W' : [0.], 'x' : [0.], 'y' : [0.], 'z' : [0.], 'n_stars' : [1]})

#m6_reference_frame = m6[['x', 'y', 'z', 'U', 'V', 'W']].median().values

In [7]:
m6 = StarClusterData(m6, data_name = 'M6', min_size = 0, max_size = 12, color = 'cyan', opacity = 1.)
sun_data = StarClusterData(sun, data_name = 'Sun', min_size = 0, max_size = 6, color = 'yellow', opacity = 1.)

sc_m6 = StarClusterCollection([m6, sun_data])
plot_3d = StarClusters3DPlotter(data_collection = sc_m6)

save_name = '/Users/cam/Downloads/m6_3d_plot.html'
time_int = np.round(np.arange(0., -60.5, -0.5), 1)
fig3d_m6 = plot_3d.generate_3d_plot(
    time = time_int,
    show = False, 
    save_name =save_name, 
    figure_layout=layout, 
    static_traces = static_traces, 
    static_traces_times = static_traces_times, 
    focus_group = 'M6'
)

In [8]:
m6.df[['x', 'y', 'z', 'U', 'V', 'W']].median().values

array([  44.99629761, -387.09166252,  -26.99161595,  -24.9792708 ,
        -17.13778873,   -9.78807908])